In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:

Number_level = 10;


In [3]:
function reward(s, a,k,s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
    p=[sum(1-T[i,i,1,j] for i in 1:9) for j in 1:NumberUnits]/sum(sum(1-T[i,i,1,j] for i in 1:9) for j in 1:NumberUnits)   
    if s==Number_level
            r=failure_penalty +  setup_cost/NumberUnits  + normal_operation  ;
        return r
       end
        r=0
     if a==1
        if s1>=s
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation ;
         return r
        else
            return 0.0
        end
        end
    if a==2
         if s1>=s
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits ;
         return r
        else
            return 0.0
        end
        end
    if a==3
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits 
         crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
        return r
    end
    end

reward (generic function with 3 methods)

In [4]:
T=zeros(Number_level,Number_level,3,100);

In [5]:
for i in 1:100
    fullname = "TM_MATRIX/tm"*string(i);
    # generate_transition_matrix(Number_level, fullname) 
    Transition_matrix = load(fullname*".jld2","transition_matrix");
    
    T[:,:,1,i].=Transition_matrix;
    T[:,:,2,i].=Transition_matrix;
    for j in 1:10
        T[j,:,3,i].=Transition_matrix[1,:];
        end
    T[10,:,1,i].=Transition_matrix[1,:]
T[10,:,2,i].=Transition_matrix[1,:]
end

In [6]:

function generative(s, a,k, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
        if s==Number_level
        s=1
        crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
        return (sp=s, r=r)
       end
        r=0
     if a==1
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation ;
         return (sp=s, r=r)
        end
    if a==2
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits ;
             return (sp=s, r=r)
        end
    if a==3

        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r+= normal_operation  ;
       return (sp=s, r=r)

    end
    end

generative (generic function with 2 methods)

In [7]:
function ValueIteration( γ)
    Q=zeros((Number_level,3,NumberUnits))
#     print(Q)
    V= -rand(Float64, (Number_level,NumberUnits))*1000
    diff=0
    i=0
    p=rand(Float64, (3,NumberUnits))
    
      
    Threads.@threads  for k in 1:NumberUnits
        
    for i in 1:1500
    for s in 1:Number_level
       for a in 1:3
                Q[s,a,k]= (reward(s,a,k)) + γ*( sum(T[s,s1,a,k]*(V[s1,k]) for (s1) in 1:Number_level ))
        end
        p[:,k].=[exp(lamb*(sum(Q[s,1,k]-findmin(Q[s,1:3,k])[1]))),exp(lamb*(Q[s,2,k]-findmin(Q[s,1:3,k])[1])),exp(lamb*(Q[s,3,k]-findmin(Q[s,1:3,k])[1]))]

        p[:,k]=p[:,k]./sum(p[:,k])
       V[s,k]=Q[s,1,k]*p[1,k]+Q[s,2,k]*p[2,k]+Q[s,3,k]*p[3,k]
        end
        
        end
    end
    return Q
    end

ValueIteration (generic function with 1 method)

In [8]:


function decison(Q, s,n=0.2)
   a_final=[]
   if Number_level in s
#         print("hi")
        for i in 1:NumberUnits
            append!(a_final,argmax(a->Q[s[i],a,i], [2,3]))
            end
        return a_final
    end
    q0=0.0
    a_0=repeat(1:1,NumberUnits);
    for i in 1:NumberUnits
            q0+=Q[s[i],1,i]
    end
    a_final=repeat(1:1,NumberUnits);

    b=1
    for i in 1:NumberUnits

            a_final[i]=argmax(a->Q[s[i],a,i], [2,3])


            if a_final[i]==3
                b+=1
            end

    end
     
        q1=0.0
    for i in 1:NumberUnits
            q1+=Q[s[i],a_final[i],i]
    end
    if !(3 in a_final) 
        a_final=a_0
    end
    if q0>q1
            return a_0
        else
            return a_final
        end
end

decison (generic function with 2 methods)

In [9]:
using DataFrames
df= DataFrame(Units=[],n=[],s=[],m=[],f=[],lambda=[],mean=[],std=[])

0×8 DataFrame

In [10]:
using Distributed

In [11]:
# print(Q)
cost=[ 

    [0 -1600 -200 -1000]
    ]

for units in [20,30]
     for c in cost
   print(units)  
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty])   
     for lambda in [100,10,5,1,0.5, 0.1,0.05,1e-2,0.5e-2,1e-3,0.5e-3,1e-4,0.5e-4,1e-5,0.5e-5,1e-6,0.5e-6,1e-7,0.5e-7,1e-8]
        global  lamb=lambda
    global NumberUnits=units
     Q=ValueIteration(.95)
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
            r2=0.0
#     print(a)
    for i in 1:NumberUnits
#             print(generative(s[i],a[i],i,234))
         s[i],r[i]=generative(s[i],a[i],i,234);
          r2+=r[i]
            end 
#     println(r2)
#     println(s)
    s=s;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
                end 
println(units)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,normal_operation,setup_cost,maintenance_penalty,failure_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end

20[0, -1600, -200, -1000]


┌ Info:  started timer at: 2023-04-17T11:15:07.985
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


20
100.0
-73089.3609841079
3333.1151824912563


┌ Info:           35.5948977s: 35 seconds, 594 milliseconds
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:62
┌ Info:  started timer at: 2023-04-17T11:16:44.774
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


20
10.0
-73141.50698724526
3312.049443245352


...

┌ Info:           25.7273565s: 25 seconds, 727 milliseconds
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:62
┌ Info:  started timer at: 2023-04-17T12:12:14.432
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


30
5.0e-8
-68484.8646389638
3692.0523332936955


┌ Info:           25.7953848s: 25 seconds, 795 milliseconds
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:62
┌ Info:  started timer at: 2023-04-17T12:13:42.119
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:54


30
1.0e-8
-68576.37574192262
3679.031220051083
-------------------------


┌ Info:           25.6857656s: 25 seconds, 685 milliseconds
└ @ TickTock C:\Users\vbansal5\.julia\packages\TickTock\fGILW\src\TickTock.jl:62


In [12]:
df

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any
1,30.0,0.0,-1600.0,-200.0,-1000.0,1.0e-8,-68576.4,3679.03
2,30.0,0.0,-1600.0,-200.0,-1000.0,5.0e-8,-68484.9,3692.05
3,30.0,0.0,-1600.0,-200.0,-1000.0,1.0e-7,-68512.8,3729.79
4,30.0,0.0,-1600.0,-200.0,-1000.0,5.0e-7,-68488.8,3708.01
5,30.0,0.0,-1600.0,-200.0,-1000.0,1.0e-6,-68563.8,3731.61
6,30.0,0.0,-1600.0,-200.0,-1000.0,5.0e-6,-68485.3,3642.15
7,30.0,0.0,-1600.0,-200.0,-1000.0,1.0e-5,-68461.6,3699.59
8,30.0,0.0,-1600.0,-200.0,-1000.0,5.0e-5,-68599.3,3672.27
9,30.0,0.0,-1600.0,-200.0,-1000.0,0.0001,-68523.4,3725.72


In [13]:
using CSV
CSV.write("ACW-MDP-HETERO.csv", df)

"ACW-MDP-HETERO.csv"